<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/deepfakes/voice_cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Voice Cloning
## Import bibliotek

In [1]:
%tensorflow_version 1.x
import os 
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CorentinJ/Real-Time-Voice-Cloning.git'
project_name = splitext(basename(git_repo_url))[0]

if not exists(project_name):
  # Sklonowanie i instalacja 
  !git clone -q --recursive {git_repo_url}

  # Instalacja dependencji
  !cd {project_name} && pip install -q -r requirements.txt
  !pip install -q gdown
  !apt-get install -qq libportaudio2
  !pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

  # Pobranie przetrenowanego modelu
  !cd {project_name} && gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc && unzip pretrained.zip

import sys
sys.path.append(project_name)

from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from dl_colab_notebooks.audio import record_audio, upload_audio

from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path

TensorFlow 1.x selected.
     |████████████████████████████████| 412.3MB 38kB/s 
     |████████████████████████████████| 686kB 54.5MB/s 
     |████████████████████████████████| 245kB 47.9MB/s 
     |████████████████████████████████| 76.6MB 39kB/s 
     |████████████████████████████████| 204kB 42.9MB/s 
     |████████████████████████████████| 286kB 53.6MB/s 
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 144579 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1) ...
Setting up libportaudio2:amd64 (19.6.0-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

Downloading...
From: https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
To: /content/Real-Time-Voice-Cloning/pretrained.zip
384MB [00:02, 182MB/s]
Archive:  pretrained.zip
   

Real-Time-Voice-Cloning/encoder/audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


## Załadowanie modeli

In [2]:
encoder.load_model(project_name / Path('encoder/saved_models/pretrained.pt'))
synthesizer = Synthesizer(project_name / Path('synthesizer/saved_models/logs-pretrained/taco_pretrained'))
vocoder.load_model(project_name / Path('vocoder/saved_models/pretrained/pretrained.pt'))

Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt


## Załadowanie audio

In [ ]:
SAMPLE_RATE = 22050

embedding = None
def _compute_embedding(audio):
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  global embedding
  embedding = None
  embedding = encoder.embed_utterance(encoder.preprocess_wav(audio, SAMPLE_RATE))

clear_output()
audio = upload_audio(sample_rate=SAMPLE_RATE)
_compute_embedding(audio)

Saving Elon Musk's Legendary Commencement Speech.mp3 to Elon Musk's Legendary Commencement Speech.mp3


## Synteza

In [13]:
text = 'I am Elon Musk from outer space. I come in peace to save the world'

In [14]:
def synthesize(embed, text):
  print('Synteza audio...')

  specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode='constant')
  clear_output()
  display(Audio(generated_wav, rate=synthesizer.sample_rate, autoplay=True))

if embedding is None:
  print('Załaduj plik audio!')
else:
  synthesize(embedding, text)